本教程将讲解如何使用交叉验证（cross-validation）来更可靠地度量模型性能。

# 引言

机器学习是一个迭代过程。

你需要不断做出选择：用哪些特征、用什么模型、模型参数如何设置等。目前，我们主要通过留出验证集来评估模型质量、并做出选择。

但这种方法也有缺点。设想你有一个包含 5000 行的数据集，通常会留出约 20% 作为验证集，即 1000 行。这会引入随机性：某个模型也许恰好在这 1000 行上表现很好，但在另一组 1000 行上却不佳。

极端情况下，如果验证集只有 1 行数据的话，比较不同的模型哪个更好时很大程度上取决于运气！

通常，验证集越大，我们对模型质量的测量噪声越小、越可靠。但更大的验证集意味着训练数据更少，可能导致模型变差！

# 什么是交叉验证？

**交叉验证**通过在数据的不同子集上多次运行建模流程，得到多组模型质量的度量。

例如，我们可以把数据分成 5 份，每份占总数据的 20%。此时我们说把数据划分为了 5 个“**折（fold）**”。

![tut5_crossval](https://storage.googleapis.com/kaggle-media/learn/images/9k60cVA.png)

之后对每个折进行一次实验：
- **实验 1**：用第 1 折作为验证（留出）集，其余作为训练集，得到基于 20% 留出集的质量度量；
- **实验 2**：用第 2 折作为留出集（其余作为训练），得到第二个质量估计；
- 重复该过程，使每个折都恰好作为一次留出集。这样 100% 的数据都会在某次实验中作为留出集出现，最终得到基于全体样本行的信息的模型质量度量（尽管并非同时使用所有行）。

# 何时使用交叉验证？

交叉验证能提供更准确的模型质量估计，在需要做大量建模决策时更加重要。但其缺点是更耗时，因为需要训练多次模型（每个折一次）。

那么，综合权衡后，何时该采用哪种方式？
- **小数据集**：额外计算成本不高，建议使用交叉验证；
- **大数据集**：单一验证集通常足够，运行更高效，往往无需重复留出。

并没有明确阈值来区分大小数据集。但如果你的模型几分钟内就能跑完，那么你可能需要考虑切换到交叉验证。

或者，你也可以直接运行交叉验证，看看各次实验的得分是否接近；若相近，则单一验证集多半就够用了。

# 示例

我们继续使用上一节的数据。将输入数据加载到 `X`，将目标变量加载到 `y`。

In [ ]:

import pandas as pd

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

随后，我们定义一个管道：先用插补器填充缺失值，再用随机森林进行预测。

虽然在没有管道的情况下也可以做交叉验证，但会很麻烦！使用管道能让代码非常简洁明了。

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

使用 scikit-learn 的 [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) 获取交叉验证得分（ `cv` 参数设置折数）。

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

`scoring` 参数用于选择要报告的模型质量指标：此处我们选择负的平均绝对误差（MAE）。scikit-learn 文档提供了[完整列表](http://scikit-learn.org/stable/modules/model_evaluation.html)。

看到“负的 MAE”可能会让人意外，这是因为 scikit-learn 的评分器（scoring）遵循“分数越大越好”，像 MAE 这类“越小越好”的误差，需要取负数才能与 API 一致；在其他场景几乎看不到“负的 MAE”的表述。

通常，我们需要一个单一指标来比较不同模型，所以会对多个实验的结果取平均。

In [ ]:
print("Average MAE score (across experiments):")
print(scores.mean())

# 结论

使用交叉验证可以更好地估计模型质量，同时还能让代码更整洁，另外，我们不再需要分别维护训练集和验证集。因此，尤其在小数据集场景下，这是一个很好的改进！

# 轮到你了

在 **[下一道练习](https://www.kaggle.com/kernels/fork/3370281)** 中动手实践吧！

---




*有问题或想交流？欢迎访问[课程讨论区](https://www.kaggle.com/learn/intermediate-machine-learning/discussion)与其他学习者交流。*